# **Performance Benchmarking of Cryptographic Mechanisms**

# 1. Introdução

O objetivo do seguinte projeto é medir o tempo que AES, RSA e SHA levam para processar arquivos de diferentes tamanhos, usando uma implementação em Python para os mecanismos de encriptação/descriptação e hash.

**AES (Advanced Encryption Standard)**

O AES é um algoritmo de criptografia simétrica, ou seja, usa a mesma chave para encriptação e decriptação. Ele opera com blocos de 128 bits e suporta chaves de 128, 192 ou 256 bits. O AES é amplamente utilizado devido à sua segurança, velocidade e eficiência em hardware e software. Um dos modos mais utilizados neste algoritmo é o CBC (Cipher Block Chaining). Este divide os dados a serem criptografados em blocos de 16 bytes, sendo o primeiro dos blocos combinado com um vetor de inicialização, via XOR, antes de ser criptografado Isso permite um encadeamento de blocos, em que cada bloco de texto cifrado é usado para modificar o próximo bloco antes da criptografia. 

**RSA (Rivest-Shamir-Adleman)**

O RSA é um algoritmo de criptografia assimétrica, ou seja, usa um par de chaves: uma chave pública para encriptação e uma chave privada para decriptação.

**SHA (Secure Hash Algorithm)**

O SHA é uma família de funções de hash criptográfico desenvolvida pela NSA (National Security Agency). A sua principal função é converter dados de qualquer tamanho num tamanho fixo de bits (hash), garantindo integridade e autenticidade.

O trabalho especifica o uso do SHA-256, que gera um hash de 256 bits. É amplamente utilizado em assinaturas digitais, blockchain e verificação de integridade de arquivos.

---

# 2. Implementação 

**Geração de Arquivos**

Foram gerados, para este projeto, 100 ficheiros para cada tamanho. 

In [ ]:
import os
import random
import string

def generate_random_file(folder, filename, size):
    file_path = os.path.join("text_files", folder, filename)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Ensure directory exists
    
    random_content = ''.join(random.choices(string.ascii_letters + string.digits, k=size))
    with open(file_path, "w") as f:
        f.write(random_content)

file_sizes = [2, 4, 8, 16, 32, 64, 128, 512, 4096, 32768, 262144, 2097152]  # Adjust for RSA if needed
for size in file_sizes:
    folder_name = str(size)  # Folder named after size
    for i in range(1, 101):
        generate_random_file(folder_name, f"{size}_{i}.txt", size)

---

**Encriptação e Descriptação com AES**

Bibliotecas e módulos necessários para a implementação das funções de AES, e variáveis globais utilizadas nas mesmas:

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher,algorithms,modes
from cryptography.hazmat.primitives import padding
import os
import timeit
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

sizes = [8, 64, 512, 4096, 32768, 262144, 2097152]
results = {}
key = os.urandom(32) # key of 32 bytes (256 bits)

A função **encrypt(data,size,iv)** cria um objeto de cifra AES, no modo CBC (Cipher Block Chaining) com uma chave (key, variável global) e um vetor de inicializção (iv), para um ficheiro de tamanho size. Se os dados não forem múltiplos de 16 bytes, este adiciona preenchimento (padding PKCS7) para que fiquem com o tamanho correto. A função encripta os dados (data) e retorna o resultado.

A função **decrypt(ciphertext, size, iv)** cria um objeto de cifra AES, no mesmo modo, com a mesma chave e vetor de inicialização. Se os dados tiverem sido preenchidos anteriormente (padding), a função remove esse preenchimento após o processo. Retorna os dados originais descriptografados. 

In [ ]:
def encrypt(data, size, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    # bytes have to be a multiple of 16... if not add padding
    if size % 16 != 0:
        # Pad the data to the AES block size (128 bits)
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(data) + padder.finalize()
        ct = encryptor.update(padded_data) + encryptor.finalize()
        return ct
    else:
        ct = encryptor.update(data) + encryptor.finalize()
        return ct


def decrypt(ciphertext, size, iv):
    # if not a size multiple of 16 it will have padding to be removed
    if size % 16 != 0:
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
        decryptor = cipher.decryptor()
        padded_data = decryptor.update(ciphertext) + decryptor.finalize()
        unpadder = padding.PKCS7(128).unpadder()
        data = unpadder.update(padded_data) + unpadder.finalize()
        return data
    else:
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
        decryptor = cipher.decryptor()
        data = decryptor.update(ciphertext) + decryptor.finalize()
        return data

---

**Encriptação e Descriptação com RSA**

Módulos necessários para as funções de RSA, tal como um gerador de chave privada e chave pública, necessárias para a encriptação e decriptação em RSA.

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes

# Generate RSA Key Pair (2048-bit)
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)
public_key = private_key.public_key()

A função **encrypt_data(data)** usa uma chave pública (public_key) para criptografar os dados. Aplica OAEP (Optimal Asymmetric Encryption Padding) com SHA-256, que aumenta a segurança ao evitar ataques baseados em estrutura de texto. Retorna os dados criptografados, que só podem ser descriptografados com a chave privada (private_key) correspondente.

A função **decrypt_data(encrypted_data)** usa a chava privada (private_key) para descriptografar os dados, aplicando o mesmo sistema OAEP + SHA-256 para garantir segurança. Retorna os dados originais, isto é, os dados anteriores à criptografia.

In [ ]:
# Function to encrypt data
def encrypt_data(data):
    return public_key.encrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

# Function to decrypt data
def decrypt_data(encrypted_data):
    return private_key.decrypt(
        encrypted_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

---

**Hashing com SHA-256**

O processo hashing transforma o "input", file_data, de qualquer tamanho, num hash fixo de 256 bits, ou 64 caracteres hexadecimais. Para isto, é necessário importar a biblioteca hashlib. 

In [ ]:
import hashlib
#Gerar a hash SHA-256 
def calculate_sha256_hash(file_data):
    return hashlib.sha256(file_data).hexdigest()

---

**Cálculo de intervalos de confiança**, que nos dá uma estimativa de incerteza ao inferir sobre uma população com base numa amostra.

In [ ]:
# Calculate and return the confidence interval for the given data.
def get_confidence_interval(data, confidence=0.95):
    # Parameters: confidence (float): The confidence level (default is 0.95).
    n = len(data)
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)
    se = std_dev / np.sqrt(n)
    
    # 95% confidence, using the normal distribution approximation: the z-score is 1.96.
    z = 1.96  
    margin_error = z * se
    return (mean - margin_error, mean + margin_error)

---

**Medição do Tempo e processamento de ficheiros com AES**

Para processar todos os ficheiros de um só tamanho (100), é usada a função **processAllFiles(size)**, que mede o tempo de encriptação e decriptação de cada ficheiro, em microsegundos, que será impresso no terminal. Cada tempo será armazenado no respetivo array: se for o tempo de encriptação irá para o arrayEnc, e se for de decriptação irá para o arrayDec. No final, armazenamos no dicionário results (variável global) esses arrays, com a chave "encryption_time" e "decryption_time" para facilitar a identificação. 

In [ ]:
def processAllFiles(size):
    arrEnc = [0] * 100  # Array to store encryption times for 100 files
    arrDec = [0] * 100  # Array to store decryption times for 100 files
    for i in range(1, 101):
        file_path = os.path.join("text_files", str(size), f"{size}_{i}.txt")
        with open(file_path, "rb") as file:
            plaintext = file.read()
        iv = os.urandom(16)  # IV has to be 16 bytes

        # Encrypt and time encryption
        encrypt_timer = timeit.Timer(lambda: encrypt(plaintext, size, iv))
        enc_time = (encrypt_timer.timeit(number=100) / 100) * 1000000
        arrEnc[i - 1] = enc_time

        ciphertext = encrypt(plaintext, size, iv)
        '''out_path = os.path.join(encrypt_dir, f"{size}.bin")
        with open(out_path, "wb") as f:
            f.write(ciphertext)'''

        # Decrypt and time decryption
        decrypt_timer = timeit.Timer(lambda: decrypt(ciphertext, size, iv))
        dec_time = (decrypt_timer.timeit(number=100) / 100) * 1000000
        arrDec[i - 1] = dec_time

        '''decrypted_text = decrypt(ciphertext, size, iv)
        out_path = os.path.join(decrypt_dir, f"{size}.txt")
        with open(out_path, "w") as f:
            f.write(decrypted_text.decode('utf-8'))'''

        filename = f"{size}_{i}.txt"
        #print(f"{filename:<11} | {size:<12} | {enc_time:.9f}         | {dec_time:.9f}")

    confidenceEnc = get_confidence_interval(arrEnc)
    confidenceDec = get_confidence_interval(arrDec)
    print(f"{size} bytes:\tEncryption: ({confidenceEnc[0]:.2f}, {confidenceEnc[1]:.2f})\tDecryption: ({confidenceDec[0]:.2f}, {confidenceDec[1]:.2f})")
    # Store all times for the given size
    results[size] = {'encryption_time': arrEnc, 'decryption_time': arrDec}


A função **processUnique(file, size)** é utilizada para a medição de um só ficheiro, 100 vezes, para se observar as variações de tempo de um só ficheiro a cada encriptação ou decriptação. Armazenamos assim cada tempo de encriptação no arrayEnc e cada tempo de decriptação no arrayDec, e no fim é criado um gráfico de barras com os resultados, exposto no capítulo 3, **Resultados**.

In [ ]:
def processUnique(file,size):
    file_path = os.path.join("text_files", str(size), file)
    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"Error: File {file_path} not found.")
        return
    # Read the file only once
    with open(file_path, "rb") as f:
        data = f.read()
    # Arrays to store encryption and decryption times
    arrayEnc = []
    arrayDec = []
    print(f"{file:<8}:")
    for i in range(100):
        with open(file_path, "rb") as f:
            plaintext = f.read()
        iv = os.urandom(16)  # IV has to be 16 bytes
        # Encrypt and time encryption
        encrypt_timer = timeit.Timer(lambda: encrypt(plaintext, size, iv))
        enc_time = (encrypt_timer.timeit(number=1000) / 1000) * 1000000
        arrayEnc.append(enc_time)

        ciphertext = encrypt(plaintext, size, iv)
        '''out_path = os.path.join(encrypt_dir, f"{size}.bin")
        with open(out_path, "wb") as f:
            f.write(ciphertext)'''

        # Decrypt and time decryption
        decrypt_timer = timeit.Timer(lambda: decrypt(ciphertext, size, iv))
        dec_time = (decrypt_timer.timeit(number=1000) / 1000) * 1000000
        arrayDec.append(dec_time) 

        ''' decrypted_text = decrypt(ciphertext, size, iv)
        out_path = os.path.join(decrypt_dir, f"{size}.txt")
        with open(out_path, "w") as f:
            f.write(decrypted_text.decode('utf-8'))'''

    plot_results(arrayEnc, arrayDec, file)

---

**Medição do Tempo e processamento de ficheiros com RSA**

A função **process_all_files(size)**, usa o mesmo método que a função para AES, mas sem o vetor de inicialização. 

In [ ]:
# Process all files in each size folder
def process_all_files(size):
    arrayEnc = [0]*100
    arrayDec = [0]*100
    results[size] = {'encryption_time': [], 'decryption_time': []}
    for i in range(1,101):
        file_path = os.path.join("text_files", str(size), str(size)+"_"+str(i)+".txt")

        if os.path.isfile(file_path):
            with open(file_path, "rb") as f:
                data = f.read()

            # Measure encryption time
            enc_time = (timeit.timeit(lambda: encrypt_data(data), number=100) / 100)* 1_000_000 
            encrypted_data = encrypt_data(data)
            arrayEnc[i-1] = enc_time
            # Measure decryption time
            dec_time = (timeit.timeit(lambda: decrypt_data(encrypted_data), number=100) / 100)* 1_000_000
            arrayDec[i-1] = dec_time
            # Store results
            results[size]['encryption_time'].append(enc_time)
            results[size]['decryption_time'].append(dec_time)
            filename = str(size) + "_" + str(i) + ".txt"
            #print(f"{filename:<8} | {size:<12} | {enc_time:.9f}         | {dec_time:.6f}")
    
    confidenceEnc = get_confidence_interval(arrayEnc)
    confidenceDec = get_confidence_interval(arrayDec)
    print(f"{size} bytes:\tEncryption: ({confidenceEnc[0]:.2f}, {confidenceEnc[1]:.2f})\tDecryption: ({confidenceDec[0]:.2f}, {confidenceDec[1]:.2f})")

A função **process_unique(file, size)** tem o mesmo efeito que a função processUnique para AES, imprimindo no terminal o tempo que o mesmo ficheiro demora a criptografar e descriptografar, para cada iteração. Após isso é retornado um gráfico de barras com os resultados, também acessível no Capítulo 3, **Resultados**.

In [ ]:
def process_unique(file,size):
    file_path = os.path.join("text_files", str(size), file)
    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"Error: File {file_path} not found.")
        return
    # Read the file only once
    with open(file_path, "rb") as f:
        data = f.read()
    # Arrays to store encryption and decryption times
    arrayEnc = []
    arrayDec = []
    print(f"{file:<8}:")
    for i in range(100):  # Run 100 times for accuracy
        try:
            # Measure encryption time
            enc_time = (timeit.timeit(lambda: encrypt_data(data), number=100) / 100)* 1_000_000
            encrypted_data = encrypt_data(data)
        except Exception as e:
            print(f"Error encrypting {file}: {e}")
            continue
        arrayEnc.append(enc_time)
        try:
            # Measure decryption time
            dec_time = (timeit.timeit(lambda: decrypt_data(encrypted_data), number=100) / 100)* 1_000_000
        except Exception as e:
            print(f"Error decrypting {file}: {e}")
            continue
        arrayDec.append(dec_time)
    # Print averaged results
        print(f"Encryption Time: {arrayEnc[i]:.9f} s | Decryption Time: {arrayDec[i]:.6f} s")
    
    plot_results(arrayEnc, arrayDec, file)

---

**Medição do Tempo e processamento de ficheiros com SHA**

A função **process_files(base_dir, size)**, semelhante às funções processAllFiles (AES) e process_all_files (RSA), calcula e retorna o tempo de hashing de 100 ficheiros de cada tamanho, em microssegundos. É nos disponibilizado, também, o intervalo de confiança para os resultados obtidos.

In [ ]:
#Processar os ficheiros de um determinado tamanho
def process_files(base_dir, size):
    size_dir = os.path.join(base_dir, str(size))
    hash_times = []
    
    for i in range(1, 101):
        file_path = os.path.join(size_dir, f"{size}_{i}.txt")
        if os.path.exists(file_path):
            with open(file_path, "rb") as file:
                file_data = file.read()
                
                timer = timeit.Timer(lambda: calculate_sha256_hash(file_data))
                hash_time = (timer.timeit(number=100) / 100) * 1000000
                hash_times.append(hash_time)

    confidenceHash = get_confidence_interval(hash_times)
    print(f"{size} bytes:\tHashing: ({confidenceHash[0]:.2f}, {confidenceHash[1]:.2f})")

    return sum(hash_times)/len(hash_times) if hash_times else None

A função **process_unique(file_name, size)**, da mesma forma que as funções para AES e RSA, calcula e retorna o tempo de hashing de um só ficheiro iterado 100 vezes, em microssegundos. Para além disso, retorna um gráfico de barras associado aos resultados obtidos. 

---

**Funções de projeção dos resultados em gráficos (AES, RSA, SHA)**

As funções são semelhantes, logo serão generalizadas para os três modos.

A função **plot_results(arrayEnc, arrayDec, file)** retorna um gráfico de barras com os tempos de encriptação e decriptação respetivos a cada iteração de um só ficheiro. O tempo de encriptação está a azul, e o tempo de decriptação a vermelho. O tempo é medido em microsegundos. Também é possível ter acesso ao desvio padrão (std) dos resultados obtidos. A função apresentada é para o modo AES, sendo igual nos restantes modos.

In [ ]:
def plot_results(arrayEnc, arrayDec, file):
    # Create a figure
    plt.figure(figsize=(12, 8))

    # Set the width of the bars
    bar_width = 0.35
    index = range(1, len(arrayEnc) + 1)  # Indices for X-axis
    
    # Plot the encryption and decryption times as bar charts
    plt.bar([i - bar_width / 2 for i in index], arrayEnc, bar_width, label='Encryption Time', color='blue')
    plt.bar([i + bar_width / 2 for i in index], arrayDec, bar_width, label='Decryption Time', color='red', alpha=0.6)

    # Compute standard deviation for encryption times
    std_enc = np.std(arrayEnc, ddof=1)  # ddof=1 for sample std deviation
    std_dec = np.std(arrayDec, ddof=1)  # ddof=1 for sample std deviation

    # Adding labels and title
    plt.xlabel('Iteration')
    plt.ylabel('Time (microseconds)')
    plt.title(f"AES Encryption and Decryption Times for {file}")
    
    # Set X-axis labels to the index of iterations
    tick_positions = range(0, 101, 10)  # Show ticks every 10 iterations
    plt.xticks(tick_positions)  # Set X-axis to display only 10th iterations
    plt.legend(loc='upper right')


    # Get the max x-axis value for proper alignment
    max_x = len(arrayEnc)  # Assuming both lists have the same length
    y_offset = max(arrayEnc) * 0.1  # Adjust vertical spacing dynamically

    # Display both standard deviations below the title and above the bars
    plt.text(0.25, 1.05, f"Std Dev (Encryption): {std_enc:.2f}", fontsize=12, color='blue',
             verticalalignment='bottom', horizontalalignment='center', transform=plt.gca().transAxes)
    
    plt.text(0.75, 1.05, f"Std Dev (Decryption): {std_dec:.2f}", fontsize=12, color='red',
             verticalalignment='bottom', horizontalalignment='center', transform=plt.gca().transAxes)
    # Display the plot
    plt.show()

A função **plot_graph(results)**, especificada para o modo AES, é utilizada para retornar o gráfico de barras para um tamanho específico, com as variações dos 100 ficheiros correspondentes. É possível, também, observar o desvio padrão dos resultados.

In [ ]:
def plot_graph(results):
    # Loop através dos resultados para cada pasta/tamanho
    for folder_size, times in results.items():
        # Criar um novo gráfico para cada tamanho de pasta
        plt.figure(figsize=(12, 8))  # Definir o tamanho da figura

        # Eixo X: Índices dos arquivos
        x_axis = range(1, len(times['encryption_time']) + 1)  # Índice dos arquivos (1 a N)

        # Largura da barra
        bar_width = 0.35
        
        # Plotando o tempo de encriptação
        plt.bar([i - bar_width / 2 for i in x_axis], times['encryption_time'], bar_width, label='Encryption Time', color='blue')
        
        # Plotando o tempo de decriptação
        plt.bar([i + bar_width / 2 for i in x_axis], times['decryption_time'], bar_width, label='Decryption Time', color='red', alpha=0.6)
        
        # Calculate the standard deviation for encryption and decryption times
        std_enc = np.std(times['encryption_time'], ddof=1)
        std_dec = np.std(times['decryption_time'], ddof=1)

        # Ajustar os rótulos e título para o gráfico
        plt.xlabel('File Index')
        plt.ylabel('Time (Microseconds)')
        plt.title(f"Encryption and Decryption Times for AES ({folder_size} bytes)")
        
        # Definir os ticks no eixo X para mostrar a cada 10 arquivos
        tick_positions = range(0, 101, 10)  # Mostrar ticks a cada 10 iterações
        plt.xticks(tick_positions)
        
        # Adicionar a legenda
        plt.legend()

        # Display both standard deviations below the title and above the bars
        plt.text(0.25, 1.05, f"Std Dev (Encryption): {std_enc:.2f}", fontsize=12, color='blue',
                verticalalignment='bottom', horizontalalignment='center', transform=plt.gca().transAxes)
        
        plt.text(0.75, 1.05, f"Std Dev (Decryption): {std_dec:.2f}", fontsize=12, color='red',
                verticalalignment='bottom', horizontalalignment='center', transform=plt.gca().transAxes)

        # Exibir o gráfico
        plt.show()

A função **plot_graph_avg(results)**, utilizada para o ficheiro de RSA, tem como retorno o gráfico de barras para a média de cada tamanho utilizado pelo modo. No caso do RSA, é nos dada a média de tempo de encriptação e decriptação para os tamanhos 2, 4, 8, 16, 32, 64 e 128 bytes. Para cada média de tamanhos também indicado o desvio padrão, tanto de encriptação como de decriptação.

In [ ]:
def plot_graph_avg(results): # works well, corrigir avg dec 
    # For each size, compute the average encryption and decryption time
    sizes = sorted(results.keys())
    avg_enc_times = []
    avg_dec_times = []
    std_enc_times = []
    std_dec_times = []
    
    for s in sizes:
        avg_enc = sum(results[s]['encryption_time']) / len(results[s]['encryption_time'])
        avg_dec = sum(results[s]['decryption_time']) / len(results[s]['decryption_time'])
        # Calculate the standard deviation for encryption and decryption times
        std_enc = np.std(results[s]['encryption_time'], ddof=1)
        std_dec = np.std(results[s]['decryption_time'], ddof=1)
        avg_enc_times.append(avg_enc)
        avg_dec_times.append(avg_dec)
        std_enc_times.append(std_enc)
        std_dec_times.append(std_dec)

    plt.figure(figsize=(12, 8))

    # Set the width of the bars
    bar_width = 0.35
    index = range(len(sizes))  # Indices for X-axis
    
    # Plot the bars for encryption and decryption times
    plt.bar([i - bar_width / 2 for i in index], avg_enc_times, bar_width, label='Encryption Time', color='blue')
    plt.bar([i + bar_width / 2 for i in index], avg_dec_times, bar_width, label='Decryption Time', color='red')
    
    # Display the standard deviation as text on the plot (just like in the other function)
    y_offset = max(max(avg_enc_times), max(avg_dec_times)) * 0.1  # Adjust vertical spacing

    # Add standard deviation text for encryption and decryption times
    for i, (std_enc, std_dec) in enumerate(zip(std_enc_times, std_dec_times)):
        plt.text(i - bar_width / 2, avg_enc_times[i] + y_offset, f"Std Dev: {std_enc:.2f}", 
                 fontsize=10, color='blue', verticalalignment='bottom', horizontalalignment='center')
        plt.text(i + bar_width / 2, avg_dec_times[i] + y_offset, f"Std Dev: {std_dec:.2f}", 
                 fontsize=10, color='red', verticalalignment='bottom', horizontalalignment='center')

    # Adding labels and title
    plt.xlabel('Size (Bytes)')
    plt.ylabel('Time (Microseconds)')
    plt.title("Average Encryption and Decryption Times for RSA")

    # Set the X-axis positions to the indices of sizes
    plt.xticks(index, sizes)  # Use the actual size values as x-axis labels

    # Display the bars and the legend
    plt.legend()
    plt.show()

---

# 3. Resultados

**AES**: Tempo de encriptação/descriptação vs. tamanho do arquivo

Em baixo estão apresentados alguns gráficos do modo AES.

Comparação entre tamanhos, usando a função processAllFiles, é apresentada a variação de tempo para os 100 ficheiros de um tamanho, com o seu desvio padrão:

![Gráfico de barras AES para o tamanho 8 bytes](gráficos/AES_8bytes.png)

![Gráfico de barras AES para 4096 bytes](gráficos/AES_4096bytes.png)

![Gráfico de barras AES para 2097152 bytes](gráficos/AES_2097152bytes.png)

Variação de tempo para um só ficheiro (100 iterações), e o seu desvio padrão:

![Gráfico de barras AES para o ficheiro 64_5.txt](gráficos/AES_uniqueFile.png)

Comparação entre a média de tempo de encriptação e decriptação de cada tamanho, com o devido desvio padrão:

![Gráfico de barras médio AES](gráficos/AES_avgBar.png)

---

**RSA**: Tempo de encriptação vs. tempo de descriptação

Em baixo estão apresentados alguns gráficos do modo RSA.

Comparação entre tamanhos, usando a função process_all_files, é apresentada a variação de tempo para os 100 ficheiros de um tamanho, com o seu desvio padrão:

![Gráfico de Barras RSA para 2 bytes](gráficos/RSA_2bytes.png)

![Gráfico de barras RSA para 16 bytes](gráficos/RSA_16bytes.png)

![Gráfico de Barras RSA para 128 bytes](gráficos/RSA_128bytes.png)

Variação de tempo para um só ficheiro (100 iterações), e o seu desvio padrão:

![Gráfico de Barras RSA para o ficheiro 64_5.txt](gráficos/RSA_uniqueFile.png)

Comparação entre a média de tempo de encriptação e decriptação de cada tamanho, com o devido desvio padrão:

![Gráfico de Barras RSA médio](gráficos/RSA_avgBar.png)

---

**SHA**: Tempo de hashing vs. tamanho do arquivo

---

# 4. Análise de Resultados

**Análise de AES**

Em relação à comparação entre ficheiros de tamanhos diferentes, podemos ver uma diferença grande. Com 8 bytes, o tempo de encriptação e decriptação é quase idêntico e muito baixo, rondando os 10 microssegundos, tal como o desvio padrão respetivo (1,65 e 1,56 microssegundos). Nos últimos ficheiros desse tamanho, tal como nos primeiros 5, ocorreu um aumento significativo, talvez devido a oscilações no processamento. Com 4096 bytes, o tempo de encriptação é maior que o tempo de decriptação, sendo ambos estáveis e previsíveis, e com um desvio padrão baixo. O tempo de processamento é muito semelhante ao de 8 bytes, principalmente na decriptação, entre 10 e 16 microssegundos, sugerindo que em ficheiros pequenos e médios, o overhead (tempo extra que um sistema precisa para executar uma tarefa) fixo da encriptação/decriptação é dominante em relação ao tamanho real dos dados.
Com 2097152 bytes, ambos os tempos aumentam significativamente, com um pico na encriptação de quase 4000 microssegundos, concluindo que não cresce linearmente com o tamanho do ficheiro.  O tempo de encriptação é bastante superior ao de decriptacao, com um aumento de mais de 2000 microssegundos entre os mesmos, indicando que os processos envolvidos na encriptação podem ser mais exigentes que na decriptação.

Usando este processo apenas num ficheiro, neste caso com 64 bytes, criptografado e descriptografado 100 vezes, a variação é quase nula, tanto para a encriptação como para a decriptação. Na imagem relativa a este processo, a primeira iteração demorou mais que as outras, fazendo com que o desvio padrão de encriptação seja maior que na decriptação, com 0.48 microssegundos.

No modo AES, pequenos ficheiros, como por exemplo 8 bytes e 32768 bytes, possuem um tempo de encriptação e decriptação extremamente rápido, com menos de 500 microssegundos, sendo praticamente instantâneas, independentemente do tamanho (quando é pequeno). O seu desvio padrão é baixo, indicando que o tempo é estável e previsível. Em ficheiros grandes, como é o caso do ficheiro de 2097152 bytes, o tempo de encriptação aumenta significativamente, com mais de 3000 microssegundos, e o tempo de decriptação também aumenta, para quase 1000 microssegundos. O desvio padrão correspondente é muito maior aos anteriores, indicando uma variação no tempo de processamento. No caso da encriptação chega aos 82.14 microssegundos, e na decriptação aos 27.92 microssegundos, menor que na encriptação, sugerindo mais estabilidade.

---------------------------

**Análise de RSA**

Começando na comparação entre tamanhos, não se vê uma diferença pois são tamanhos pequenos. Como nos gráficos apresentados, para 2, 16 e 128 bytes, os tempos de encriptação e decriptação não mudam, sendo aproximadamente 100 microssegundos e 1750 microssegundos, respetivamente. Isto acontece porque o RSA não trabalha diretamente com cada byte de dados, como o AES. Em vez disso, o RSA opera em blocos do tamanho da chave (2048 bits = 256 bytes), então encriptar estes tamanhos é praticamente o mesmo, pois todos cabem no mesmo bloco. Entre encriptação e decriptação de RSA, a encriptação é mais rápida devido ao expoente público ser pequeno, o que torna os cálculos mais eficientes. Já a decriptação usa um expoente privado, que é muito maior, tornando os cálculos exponenciais muito mais caros a nível computacional. Esta exige operações modulares mais complexas e demoradas. 

No processo de apenas um ficheiro, neste caso de 64 bytes, criptografado e descriptografado 100 vezes, o tempo de cada iteração é idêntico aos tempos acima mencionados, e não há variações significativas. A encriptação é extremamente rápida e consistente, com um desvio padrão de 2.02 microssegundos, e a decriptação é significativamente mais lenta, com um desvio padrão de 7.71 microssegundos, que mesmo sendo maior que o da encriptação, continua a ser relativamente pequeno. 

Em relação à comparação do tempo médio para diferentes tamanhos no RSA, este mantém-se praticamente constante. O tempo de encriptação é de 1750 microssegundos, e o tempo de encriptação é de aproximadamente 100 microssegundos. O desvio padrão de encriptação é baixíssimo, rondando os 0.20 microssegundos, o que indica que a encriptação é muito determinística. O desvio padrão de decriptação é maior e na maioria dos casos ronda os 7 ou 8 microssegundos. Há uma ligeira variação no tempo de execução, possivelmente devido a otimizações internas de hardware. Podemos concluir que o RSA não é eficiente para dados de grande tamanho, e deve ser usado para encriptar chaves simétricas, e não ficheiros inteiros.

------

**Análise de SHA**

---


**Comparação entre AES e RSA**

Os modos AES e RSA apresentam diferenças variadas e por essa mesma razão são utilizadas com diferentes objetivos. O modo AES, por exemplo, é muito mais rápido do que o modo RSA, visto que o AES cifra em poucos microssegundos, mesmo para ficheiros de 1MB ou mais. O RSA começa logo com 100 microssegundos de encriptação e 1750 microssegundos de decriptação, e piora significativamente para dados maiores.
O modo RSA possui um tempo fixo para ficheiros pequenos, enquanto que o modo AES cresce linearmente com o tamanho do ficheiro, mas continua extremamente rápido.
Outra diferença entre os dois modos, é que o modo AES trabalha com blocos pequenos, de 16 bytes e pode cifrar dados de qualquer tamanho, enquanto que o modo RSA opera em blocos do tamanho da chave, neste caso de 256 bytes, o que se torna ineficiente para ficheiros grandes.
Pode-se concluir que o AES é mais rápido e eficiente, sendo usado para cifrar dados grandes, ao contrário do RSA, que é eficiente apenas na encriptação de pequenas quantidades de dados, como chaves AES.

**Comparação entre AES e SHA-256**

**Comparação entre RSA e SHA-256**

---

# 5. Conclusão

Através este trabalho, é possível concluir que o tamanho dos arquivos influencia diretamente o tempo de processamento dos algoritmos criptográficos. Enquanto que o RSA mantém tempos relativamente constantes em ficheiros pequenos, apresenta um crescimento significativo em relação ao temlo de processamento de ficheiros maiores, tornando-se ineficiente. Por outro lado, o AES demonstra um desemnho altamente linear, mantendo tempos de encriptação e decriptação significativamente pequenos, mesmo em tamanhos grandes. 
SHA...
Os resultados obtidos confirmam, então, que os algoritmos simétricos, como o AES, são mais eficientes para encriptação de grandes quantidades de dados, enquanto que algoritmos assimétricos, como o RSA, são mais adequados para encriptar pequenas quantidades de dados, como chaves. 
Esta diferença de desempenho reforça o método de combinar algoritmos. 
Assim, o estudo reforça a importância da escolha adequada do algoritmo de criptografia dependendo do contexto e da necessidade, equilibrando segurança e eficiência para otimizar o desempenho dos sistemas criptográficos.